## AlexNet
图像识别一直是机器学习中非常重要的一个研究内容, 2010年开始举办的[ILSVRC](http://image-net.org/challenges/LSVRC/)比赛更是吸引了无数的团队. 这个比赛基于一个百万量级的图片数据集, 提出一个图像1000分类的挑战. 前两年在比赛中脱颖而出的都是经过人工挑选特征, 再通过`SVM`或者`随机森林`这样在过去十几年中非常成熟的机器学习方法进行分类的算法. 

在2012年, 由 [Alex Krizhevsky](https://www.cs.toronto.edu/~kriz/), [Ilya Sutskever](http://www.cs.toronto.edu/~ilya/), [Geoffrey Hinton](http://www.cs.toronto.edu/~hinton/)提出了一种使用卷积神经网络的方法, 以 [0.85](http://image-net.org/challenges/LSVRC/2012/results.html#abstract) 的`top-5`正确率一举获得当年分类比赛的冠军, 超越使用传统方法的第二名10个百分点, 震惊了当时的学术界, 从此开启了人工智能领域的新篇章.

这次的课程我们就来复现一次`AlexNet`, 首先来看它的网络结构

<img src="https://image.ibb.co/mP9C0x/alexnet_arch.png">

可以看出`AlexNet`就是几个卷积池化堆叠后连接几个全连接层, 下面就让我们来尝试仿照这个结构来解决[cifar10](https://www.cs.toronto.edu/~kriz/cifar.html)分类问题.

### `CIFAR10`

[![cifar10](https://image.ibb.co/n885Lx/cifar10.png)](https://www.cs.toronto.edu/~kriz/cifar.html)

这是一个包含60000张$32\times32$图片的数据库, 包含50000张训练集和10000张测试集, 在这里我们提供一个脚本帮助我们读取数据

In [1]:
import tensorflow as tf
#from utils import cifar10_input

In [2]:
# 我们定义一个批次有64个样本
batch_size = 64

In [3]:
# 下载数据
#data_dir = 'cifar10_data'
#cifar10_input.maybe_download(data_dir='cifar10_data')

In [4]:
# 获取训练集
# 在使用随机梯度下降法的时候, 训练集要求打乱样本
#train_imgs, train_labels = cifar10_input.inputs(eval_data=False, 
#                                                data_dir='%s/cifar-10-batches-bin/' % data_dir, 
#                                                batch_size=batch_size, 
#                                                shuffle=True)

# 获取测试集
# 测试集不需要打乱样本
#val_imgs, val_labels = cifar10_input.inputs(eval_data=True, 
#                                            data_dir='%s/cifar-10-batches-bin/' % data_dir, 
#                                            batch_size=batch_size, 
#                                            shuffle=False)

In [3]:
import cifar10_input

data_dir = "../cifar_data/cifar-10-batches-bin/"
batch_size = 64

train_imgs, train_labels = cifar10_input.distorted_inputs(data_dir=data_dir, batch_size=batch_size)

val_images, val_labels = cifar10_input.inputs(eval_data=True,data_dir="../cifar_data/cifar-10-batches-bin/",
                                               batch_size=batch_size)

Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(string_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensor_slices(input_tensor).shuffle(tf.shape(input_tensor, out_type=tf.int64)[0]).repeat(num_epochs)`. If `shuffle=False`, omit the `.shuffle(...)`.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.Dataset.from_tensors(tensor).repeat(num_epochs)`.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
To construct input pipelines, use the `tf.data` module.
Instructions for updating:
Queue-based input pipelines have been replaced by `tf.data`. Use `tf.data.FixedLengthRecordDataset`.
Filling queue with 20000 CIF

In [4]:
train_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_TRAIN # 训练样本的个数
val_examples = cifar10_input.NUM_EXAMPLES_PER_EPOCH_FOR_EVAL       # 测试样本的个数

In [5]:
# 像之前一样, 我们构造几个生成变量的函数
def variable_weight(shape, stddev=5e-2):
    init = tf.truncated_normal_initializer(stddev=stddev)
    return tf.get_variable(shape=shape, initializer=init, name='weight')

def variable_bias(shape):
    init = tf.constant_initializer(0.1)
    return tf.get_variable(shape=shape, initializer=init, name='bias')

前面的课程大家已经见过如何使用卷积和池化了, 但是由于`tensorflow`提供的接口是底层的, 为了方便, 我们写一个上层的接口来调用

In [6]:
def conv(x, ksize, out_depth, strides, padding='SAME', act=tf.nn.relu, scope='conv_layer', reuse=None):
    """构造一个卷积层
    Args:
        x: 输入
        ksize: 卷积核的大小, 一个长度为2的`list`, 例如[3, 3]
        output_depth: 卷积核的个数
        strides: 卷积核移动的步长, 一个长度为2的`list`, 例如[2, 2]
        padding: 卷积核的补0策略
        act: 完成卷积后的激活函数, 默认是`tf.nn.relu`
        scope: 这一层的名称(可选)
        reuse: 是否复用
    
    Return:
        out: 卷积层的结果
    """
    # 这里默认数据是NHWC输入的
    in_depth = x.get_shape().as_list()[-1]
    
    with tf.variable_scope(scope, reuse=reuse):
        # 先构造卷积核
        shape = ksize + [in_depth, out_depth]
        with tf.variable_scope('kernel'):
            kernel = variable_weight(shape)
            
        strides = [1, strides[0], strides[1], 1]
        # 生成卷积
        conv = tf.nn.conv2d(x, kernel, strides, padding, name='conv')
        
        # 构造偏置
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
            
        # 和偏置相加
        preact = tf.nn.bias_add(conv, bias)
        
        # 添加激活层
        out = act(preact)
        
        return out

In [7]:
def max_pool(x, ksize, strides, padding='SAME', name='pool_layer'):
    """构造一个最大值池化层
    Args:
        x: 输入
        ksize: pooling核的大小, 一个长度为2的`list`, 例如[3, 3]
        strides: pooling核移动的步长, 一个长度为2的`list`, 例如[2, 2]
        padding: pooling的补0策略
        name: 这一层的名称(可选)
    
    Return:
        pooling层的结果
    """
    return tf.nn.max_pool(x, [1, ksize[0], ksize[1], 1], [1, strides[0], strides[1], 1], padding, name=name)

In [8]:
def fc(x, out_depth, act=tf.nn.relu, scope='fully_connect', reuse=None):
    """构造一个全连接层
    Args:
        x: 输入
        out_depth: 输出向量的维数
        act: 激活函数, 默认是`tf.nn.relu`
        scope: 名称域, 默认是`fully_connect`
        reuse: 是否需要重用
    """
    in_depth = x.get_shape().as_list()[-1]
    
    # 构造全连接层的参数
    with tf.variable_scope(scope, reuse=reuse):
        # 构造权重
        with tf.variable_scope('weight'):
            weight = variable_weight([in_depth, out_depth])
            
        # 构造偏置项
        with tf.variable_scope('bias'):
            bias = variable_bias([out_depth])
        
        # 一个线性函数
        fc = tf.nn.bias_add(tf.matmul(x, weight), bias, name='fc')
        
        # 激活函数作用
        out = act(fc)
        
        return out

有了上面这些上层函数之后, 我们就可以轻松构建`AlexNet`了.

In [9]:
def alexnet(inputs, reuse=None):
    """构建 Alexnet 的前向传播
    Args:
        inpus: 输入
        reuse: 是否需要重用
        
    Return:
        net: alexnet的结果
    """
    # 首先我们声明一个变量域`AlexNet`
    with tf.variable_scope('AlexNet', reuse=reuse):
        # 第一层是 5x5 的卷积, 卷积核的个数是64, 步长是 1x1, padding是`VALID`
        net = conv(inputs, [5, 5], 64, [1, 1], padding='VALID', scope='conv1')
        
        # 第二层是 3x3 的池化, 步长是 2x2, padding是`VALID`
        net = max_pool(net, [3, 3], [2, 2], padding='VALID', name='pool1')
        
        # 第三层是 5x5 的卷积, 卷积核的个数是64, 步长是 1x1, padding是`VALID`
        net = conv(net, [5, 5], 64, [1, 1], scope='conv2')
        
        # 第四层是 3x3 的池化, 步长是 2x2, padding是`VALID`
        net = max_pool(net, [3, 3], [2, 2], padding='VALID', name='pool2')
        #print(net.get_shape())
        # 将矩阵拉长成向量
        net = tf.reshape(net, [-1, 4*4*64])
        
        # 第五层是全连接层, 输出个数为384
        net = fc(net, 384, scope='fc3')
        
        # 第六层是全连接层, 输出个数为192
        net = fc(net, 192, scope='fc4')
        
        # 第七层是全连接层, 输出个数为10, 注意这里不要使用激活函数
        net = fc(net, 10, scope='fc5', act=tf.identity)
        
        return net

通过 alexnet 构建训练和测试的输出

In [10]:
train_out = alexnet(train_imgs)

注意当再次调用 alexnet 函数时, 如果要使用之前调用时产生的变量值, **必须**要重用变量域

In [11]:
val_out = alexnet(val_images, reuse=True)

#### 定义损失函数
这里真实的 labels 不是一个 one_hot 型的向量, 而是一个数值, 因此我们使用 

In [12]:
with tf.variable_scope('loss'):
    train_loss = tf.losses.sparse_softmax_cross_entropy(labels=train_labels, logits=train_out, scope='train')
    val_loss = tf.losses.sparse_softmax_cross_entropy(labels=val_labels, logits=val_out, scope='val')

#### 定义正确率`op`

In [13]:
with tf.name_scope('accuracy'):
    with tf.name_scope('train'):
        train_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(train_out, axis=-1, output_type=tf.int32), train_labels), tf.float32))
    with tf.name_scope('train'):
        val_acc = tf.reduce_mean(tf.cast(tf.equal(tf.argmax(val_out, axis=-1, output_type=tf.int32), val_labels), tf.float32))

#### 构造训练`op`

In [14]:
lr = 0.01

opt = tf.train.MomentumOptimizer(lr, momentum=0.9)
train_op = opt.minimize(train_loss)

在这里我们已经将训练过程封装好了, 感兴趣的同学可以进入`train.py`查看

In [ ]:
from utils.learning import train

In [ ]:
train(train_op, train_loss, train_acc, val_loss, val_acc, 20000, batch_size)

可以看到, 20000步的训练后, `AlexNet`在训练集和测试集分别达到了`0.97`和`0.72`的准确率